In [1]:
% cd /home/mayu-ot/durga/Experiments/loc_iparaphrasing/

/mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing


In [56]:
from func.datasets.datasets import BBoxDataset, PreCompFeatDataset
import pandas as pd
import numpy as np
from nltk.metrics import edit_distance
import os

In [66]:
class PLCLCBBoxDataset(BBoxDataset):
    def __init__(self, split):
        df = pd.read_csv('data/pl-clc/localization_%s.csv' % split,
                         index_col=0)
        pair_df = pd.read_csv('data/phrase_pair_%s.csv' % split, index_col=0)
        df = df.drop_duplicates(['image', 'org_phrase'])
        df = df.reset_index()
        df = df.rename(columns={'org_phrase': 'phrase'})
        df = df[['image', 'phrase', 'xmin', 'ymin', 'xmax', 'ymax']]
        self.df = df
        
class PLCLCDataset(PreCompFeatDataset):
    def setup(self, split):
        self.bbox_df = PLCLCBBoxDataset(split).df
        self.vis_feat = np.load('data/region_feat/plclc_roi-frcnn/%s.npy' % split)
        
        if os.path.exists('data/plclc/vis_indices_%s.json' % split):
            vis_indices = json.load(open('data/plclc/vis_indices_%s.json' % split))
        else:
            vis_indices = prepare_vis_feat_indices(df, pair_df)
        
        self.vis_indices = vis_indices
    
    def get_nid(self, img_id, phrase):
        return self.vis_indices[str(img_id)][phrase]
    
    def get_example(self, i):
        # get indices to get visual features
        img_id = self.pair_data.at[i, 'image']
        phr1 = self.pair_data.at[i, 'original_phrase1']
        phr2 = self.pair_data.at[i, 'original_phrase2']
        nid_1 = self.bbox_df.at[(img_id, phr1), 'n_id']
        nid_2 = self.bbox_df.at[(img_id, phr2), 'n_id']
        
        if isinstance(nid_1, np.ndarray):
            nid_1 = nid_1[0]
            
        if isinstance(nid_2, np.ndarray):
            nid_2 = nid_2[0]
        
        # get language features
        xvis_1 = self.vis_feat[nid_1]
        xvis_2 = self.vis_feat[nid_2]
        
        #
        phr1 = self.pair_data.at[i, 'phrase1']
        phr2 = self.pair_data.at[i, 'phrase2']
        
        phr_1 = self._feat[self._p2i_dict[phr1]]
        phr_2 = self._feat[self._p2i_dict[phr2]]
        
        l = self.pair_data.at[i, 'ytrue']
        
        return phr_1, phr_2, xvis_1, xvis_2, l

In [67]:
def prepare_vis_feat_indices(bbox_df, pair_df):
    phr1 = pair_df[['image', 'original_phrase1']].drop_duplicates()
    phr1 = phr1.rename(index=str, columns={'original_phrase1': 'phrase'})
    phr2 = pair_df[['image', 'original_phrase2']].drop_duplicates()
    phr2 = phr2.rename(index=str, columns={'original_phrase2': 'phrase'})
    d_df = pd.concat([phr1, phr2]).drop_duplicates()

    indices = {}
    cur_img = 0
    for _, row in d_df.iterrows():
        img, phrase = row
        if img != cur_img:
            rows = bbox_df[bbox_df.image == img]
            cur_img = img
            indices.setdefault(str(img), {})

        dist = list(map(lambda x: edit_distance(phrase.lower(), x), rows.phrase))
        row = rows.iloc[np.argmin(dist)]
        index = row.name
        indices[str(img)][phrase] = int(index)

    return indices

In [68]:
bbox_data = PLCLCBBoxDataset('val')

In [50]:
split = 'val'
df = pd.read_csv('data/pl-clc/localization_%s.csv' % split,
                 index_col=0)
df = df.drop_duplicates(['image', 'org_phrase'])
df = df.reset_index()
df = df.rename(columns={'org_phrase': 'phrase'})
df.head()

,index,image,cap_id,phr_id,category,phrase,bbox,xmin,ymin,xmax,ymax,score
0,0,100652400,1,197,people,A man,114,47.749035,41.13176,104.80542,203.022903,2.425627
1,30,100652400,1,198,clothing,a blue hard hat,98,40.000000,26.00000,113.00000,138.000000,3.362853
2,60,100652400,1,199,clothing,orange safety vest,177,91.000000,102.00000,172.00000,180.000000,3.239620
3,90,100652400,1,200,scene,an intersection,22,1.000000,79.00000,375.00000,500.000000,2.365725
4,120,100652400,1,201,other,a flag,297,271.000000,259.00000,375.00000,324.000000,2.930076


In [38]:
pair_df = pd.read_csv('data/phrase_pair_%s.csv' % split, index_col=0)
pair_df.head()

,image,phrase1,phrase2,original_phrase1,original_phrase2,ytrue,roi1,roi2
0,100652400,man,hard+hat,A man,a hard hat,False,116,-1
1,100652400,man,caution+vest,A man,a caution vest,False,116,-1
2,100652400,man,street,A man,the street,False,116,21
3,100652400,man,orange+flag,A man,an orange flag,False,116,-1
4,100652400,blue+hard+hat,man,a blue hard hat,A man,False,-1,116


In [43]:
phr1 = pair_df[['image', 'original_phrase1']].drop_duplicates()
phr1 = phr1.rename(index=str, columns={'original_phrase1': 'phrase'})
phr2 = pair_df[['image', 'original_phrase2']].drop_duplicates()
phr2 = phr2.rename(index=str, columns={'original_phrase2': 'phrase'})
d_df = pd.concat([phr1, phr2]).drop_duplicates()
d_df.head()

,image,phrase
0,100652400,A man
4,100652400,a blue hard hat
9,100652400,orange safety vest
14,100652400,an intersection
19,100652400,a flag


In [39]:
row = pair_df.iloc[0][['image', 'original_phrase1', 'original_phrase2']]
img, phr1, phr2 = row
img, phr1, phr2

(100652400, 'A man', 'a hard hat')

In [40]:
sub_df = df[df.image == img]
list(map(lambda x: edit_distance(phr1, x), sub_df.org_phrase))

AttributeError: 'DataFrame' object has no attribute 'image'

In [35]:
sub_df.index

Int64Index([0, 30, 60, 90, 120, 180, 210, 240, 270, 330, 360, 420, 450, 480,
            540, 600, 690],
           dtype='int64')

In [41]:
df[img]

KeyError: 100652400